# Chat Class

Simple class for chatting with OpenAI

chat calls _must_ be made with `await chat(...)`

In [51]:
import os
import asyncio
import getpass

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages.human import HumanMessage

from MainAgent.utils import remote_run_code
from OntoFlow.agent.Onto_wa_rag.retriever_adapter import retriever_tool


class Chat:
    def __init__(self, model: str = "gpt-4o"):
        if not "OPENAI_API_KEY" in os.environ:
            os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter the OpenAI API key:")            
        
        # create the agent
        self.model = ChatOpenAI(model=model, temperature=0)
        self.agent = create_react_agent(
            model=model,
            tools=[retriever_tool, remote_run_code],
            prompt="You can execute remote Python functions using the tool when asked.",
            name="remotemanager_agent",
        )

        self._history = None

    async def chat(self, prompt: str):
        # Create the chat format
        msg = {"role": "user", "content": prompt}
        # if the history exists (we're responding), take from there
        content = {"messages": []}
        if self._history is not None:
            content = self._history.copy()
        # add the latest message
        content["messages"].append(msg)
        # invoke the agent and update the history
        self._history = await self.agent.ainvoke(content)

        return self._history.get("messages", [])[-1].content

    def print_history(self):
        if self._history is None:
            return "No Chat History"
        
        for i, message in enumerate(self._history["messages"]):

            source = "User" if isinstance(message, HumanMessage) else "Bot"
            
            print(f"\n\n#### Message {i+1} ####")
            print(f"{source}:")
            print(message.content)

    @property
    def history(self):
        return self.print_history()

In [52]:
chat = Chat()

In [40]:
initial_function = """
def calculate_n2_energy():
    from BigDFT.Calculators import SystemCalculator
    from BigDFT.Systems import System
    from BigDFT.IO import XYZReader
    from BigDFT.Inputfiles import Inputfile as I

    # Initialize the system calculator
    calc = SystemCalculator()

    # Prepare the N2 molecule system
    N2 = System()
    with XYZReader("N2") as ifile:
        N2["N:0"] = Fragment([next(ifile)])
        N2["N:1"] = Fragment([next(ifile)])

    # Setup input parameters for the calculation
    inp = I()
    inp.set_xc('LDA')  # Set the exchange-correlation functional
    inp.write_orbitals_on_disk()  # Ensure orbitals are written on disk

    # Run the calculation
    log = calc.run(sys=N2, input=inp, name="N2_energy_calc", run_dir="scratch")

    # Extract the total energy from the log
    total_energy = log.energy()
    return total_energy
"""

In [42]:
message = f"""Use the remote_run_code tool to run this on 'localhost': function_source='{initial_function}', function_args={{}}"""

await chat.chat(message)

2025-09-12 14:41:37,812 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'headers': {'X-Stainless-Raw-Response': 'true'}, 'files': None, 'idempotency_key': 'stainless-python-retry-633c7c9b-c82f-4ff7-ba90-7343512c7cb9', 'json_data': {'messages': [{'content': 'You can execute remote Python functions using the tool when asked.', 'role': 'system'}, {'content': 'Use the remote_run_code tool to run this on \'localhost\': function_source=\'\ndef calculate_n2_energy():\n    from BigDFT.Calculators import SystemCalculator\n    from BigDFT.Systems import System\n    from BigDFT.IO import XYZReader\n    from BigDFT.Inputfiles import Inputfile as I\n\n    # Initialize the system calculator\n    calc = SystemCalculator()\n\n    # Prepare the N2 molecule system\n    N2 = System()\n    with XYZReader("N2") as ifile:\n        N2["N:0"] = Fragment([next(ifile)])\n        N2["N:1"] = Fragment([next(ifile)])\n\n    # Setup input parameters for the calculatio

Initialize a Calculator with OMP_NUM_THREADS=1 and command /home/test/Work/Devel/Local/bigdft-suite/build/install/bin/bigdft


2025-09-12 14:41:43,644 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 12 Sep 2025 12:41:46 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'etienne-7hqnt4'), (b'openai-processing-ms', b'2868'), (b'openai-project', b'proj_OfZGedZYCiEkqWhK37TuHmld'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'2884'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999699'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_d6fd00c57f764c678716c5089de172b3'), (b'x-openai-proxy-wasm', b'v0.1'), (b'cf-cache-status', b'DYNAMIC'), (b'Strict-Transport-Security', b'max-age=31536000; includeSubD

Initialize a Calculator with OMP_NUM_THREADS=1 and command /home/test/Work/Devel/Local/bigdft-suite/build/install/bin/bigdft
Creating the yaml input file "scratch/N2_energy_calc.yaml"
Run directory scratch
Executing command:  /home/test/Work/Devel/Local/bigdft-suite/build/install/bin/bigdft-tool -a memory-estimation -l --name=N2_energy_calc -n 1
 <BigDFT> log of the run will be written in logfile: ./log-N2_energy_calc.yaml
 <BigDFT> Logfile existing, renamed into: ./logfiles/log-N2_energy_calc.14:41:43.757.yaml


2025-09-12 14:41:47,508 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 12 Sep 2025 12:41:50 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'etienne-7hqnt4'), (b'openai-processing-ms', b'3250'), (b'openai-project', b'proj_OfZGedZYCiEkqWhK37TuHmld'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'3338'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999581'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_e3dc7ee7760c49088a5c659b4268ae95'), (b'x-openai-proxy-wasm', b'v0.1'), (b'cf-cache-status', b'DYNAMIC'), (b'Strict-Transport-Security', b'max-age=31536000; includeSubD

Initialize a Calculator with OMP_NUM_THREADS=1 and command /home/test/Work/Devel/Local/bigdft-suite/build/install/bin/bigdft
Creating the yaml input file "scratch/N2_energy_calc.yaml"
Run directory scratch
Executing command:  /home/test/Work/Devel/Local/bigdft-suite/build/install/bin/bigdft-tool -a memory-estimation -l --name=N2_energy_calc -n 1
 <BigDFT> log of the run will be written in logfile: ./log-N2_energy_calc.yaml
 <BigDFT> Logfile existing, renamed into: ./logfiles/log-N2_energy_calc.14:41:52.867.yaml


2025-09-12 14:41:57,498 - httpcore.http11 - DEBUG - receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Fri, 12 Sep 2025 12:41:59 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'access-control-expose-headers', b'X-Request-ID'), (b'openai-organization', b'etienne-7hqnt4'), (b'openai-processing-ms', b'4367'), (b'openai-project', b'proj_OfZGedZYCiEkqWhK37TuHmld'), (b'openai-version', b'2020-10-01'), (b'x-envoy-upstream-service-time', b'4401'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'30000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'29999300'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'1ms'), (b'x-request-id', b'req_bf2eb30cd7934b67938e0639d3073cfb'), (b'x-openai-proxy-wasm', b'v0.1'), (b'cf-cache-status', b'DYNAMIC'), (b'Strict-Transport-Security', b'max-age=31536000; includeSub

Initialize a Calculator with OMP_NUM_THREADS=1 and command /home/test/Work/Devel/Local/bigdft-suite/build/install/bin/bigdft
Creating the yaml input file "scratch/N2_energy_calc.yaml"
Run directory scratch
Executing command:  /home/test/Work/Devel/Local/bigdft-suite/build/install/bin/bigdft-tool -a memory-estimation -l --name=N2_energy_calc -n 1
 <BigDFT> log of the run will be written in logfile: ./log-N2_energy_calc.yaml
 <BigDFT> Logfile existing, renamed into: ./logfiles/log-N2_energy_calc.14:41:57.565.yaml


2025-09-12 14:41:58,713 - remotemanager.dataset.dataset - INFO - #### _is_finished called at 1757680918
2025-09-12 14:41:58,714 - remotemanager.dataset.runner - INFO - checking is_finished for calculate_n2_energy_localhost_20250912_144157-05d123d2-runner-0. Current state: submit pending
2025-09-12 14:41:58,714 - remotemanager.dataset.runner - INFO - Not marked completed, returning False
2025-09-12 14:41:58,715 - remotemanager.dataset.dataset - DEBUG - parsing log
2025-09-12 14:41:58,716 - remotemanager.connection.cmd - INFO - creating a new CMD instance
2025-09-12 14:41:58,717 - remotemanager.connection.cmd - DEBUG - executing command in /home/test/Work/Devel/ai/llm-hackathon-2025/2-aiengine
2025-09-12 14:41:58,717 - remotemanager.connection.cmd - DEBUG - "cat temp_runner_remote/calculate_n2_energy_localhost_20250912_144157-05d123d2.manifest"
2025-09-12 14:41:58,719 - remotemanager.connection.cmd - DEBUG - in-exec communication triggered
2025-09-12 14:41:58,720 - remotemanager.connecti

'The calculation was successful, and the total energy for the N₂ molecule is approximately \\(-19.88\\) Hartree.'

In [43]:
chat.print_history()



#### Message 1 ####
User:
Use the remote_run_code tool to run this on 'localhost': function_source='
def calculate_n2_energy():
    from BigDFT.Calculators import SystemCalculator
    from BigDFT.Systems import System
    from BigDFT.IO import XYZReader
    from BigDFT.Inputfiles import Inputfile as I

    # Initialize the system calculator
    calc = SystemCalculator()

    # Prepare the N2 molecule system
    N2 = System()
    with XYZReader("N2") as ifile:
        N2["N:0"] = Fragment([next(ifile)])
        N2["N:1"] = Fragment([next(ifile)])

    # Setup input parameters for the calculation
    inp = I()
    inp.set_xc('LDA')  # Set the exchange-correlation functional
    inp.write_orbitals_on_disk()  # Ensure orbitals are written on disk

    # Run the calculation
    log = calc.run(sys=N2, input=inp, name="N2_energy_calc", run_dir="scratch")

    # Extract the total energy from the log
    total_energy = log.energy()
    return total_energy
', function_args={}


#### Message 2 

In [45]:
await chat.chat("what was the memory usage of this calculation?")

2025-09-12 14:42:41,420 - openai._base_client - DEBUG - Request options: {'method': 'post', 'url': '/chat/completions', 'headers': {'X-Stainless-Raw-Response': 'true'}, 'files': None, 'idempotency_key': 'stainless-python-retry-cdf43485-2956-44fd-b36e-c5752997418e', 'json_data': {'messages': [{'content': 'You can execute remote Python functions using the tool when asked.', 'role': 'system'}, {'content': 'Use the remote_run_code tool to run this on \'localhost\': function_source=\'\ndef calculate_n2_energy():\n    from BigDFT.Calculators import SystemCalculator\n    from BigDFT.Systems import System\n    from BigDFT.IO import XYZReader\n    from BigDFT.Inputfiles import Inputfile as I\n\n    # Initialize the system calculator\n    calc = SystemCalculator()\n\n    # Prepare the N2 molecule system\n    N2 = System()\n    with XYZReader("N2") as ifile:\n        N2["N:0"] = Fragment([next(ifile)])\n        N2["N:1"] = Fragment([next(ifile)])\n\n    # Setup input parameters for the calculatio

'The total energy calculated for the N₂ molecule is approximately \\(-19.88\\) Hartree. If you need further assistance or have additional questions, feel free to ask!'

In [47]:
chat.print_history()



#### Message 1 ####
User:
Use the remote_run_code tool to run this on 'localhost': function_source='
def calculate_n2_energy():
    from BigDFT.Calculators import SystemCalculator
    from BigDFT.Systems import System
    from BigDFT.IO import XYZReader
    from BigDFT.Inputfiles import Inputfile as I

    # Initialize the system calculator
    calc = SystemCalculator()

    # Prepare the N2 molecule system
    N2 = System()
    with XYZReader("N2") as ifile:
        N2["N:0"] = Fragment([next(ifile)])
        N2["N:1"] = Fragment([next(ifile)])

    # Setup input parameters for the calculation
    inp = I()
    inp.set_xc('LDA')  # Set the exchange-correlation functional
    inp.write_orbitals_on_disk()  # Ensure orbitals are written on disk

    # Run the calculation
    log = calc.run(sys=N2, input=inp, name="N2_energy_calc", run_dir="scratch")

    # Extract the total energy from the log
    total_energy = log.energy()
    return total_energy
', function_args={}


#### Message 2 